In [8]:
!pip install requests beautifulsoup4 pandas PyPDF2 pytesseract langchain langchain_community crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.3/209.3 kB 3.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.1 MB/s eta 0:00:00
   ━━

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PyPDF2 import PdfReader
import pytesseract
from PIL import Image
import io
from langchain.llms import OpenAI

# Ensure you have Tesseract OCR installed and its path set correctly
pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'


ModuleNotFoundError: No module named 'langchain_community'

In [4]:
# Function to scrape HTML content
def scrape_html(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    projects = []
    # Adjust based on the specific structure of the website
    for project in soup.find_all('div', class_='project-item'):
        title = project.find('h2').text
        description = project.find('p', class_='description').text
        deadline = project.find('span', class_='deadline').text
        projects.append({'Title': title, 'Description': description, 'Deadline': deadline})
    return projects

# Function to extract text from PDF
def extract_pdf_text(url):
    response = requests.get(url)
    pdf = PdfReader(io.BytesIO(response.content))
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

# Function to extract text from images using OCR
def extract_image_text(url):
    response = requests.get(url)
    img = Image.open(io.BytesIO(response.content))
    text = pytesseract.image_to_string(img)
    return text

# Function to summarize text using LLM
def summarize_text(text):
    prompt = f"Summarize the following text: {text}"
    llm = OpenAI(api_key='your_openai_api_key')
    summary = llm.generate(prompt)
    return summary


# Claude

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PyPDF2 import PdfReader
import pytesseract
from PIL import Image
import io
from langchain.llms import OpenAI
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.prompts import StringPromptTemplate
from langchain import LLMChain
from crewai import Agent, Task, Crew

In [10]:
def summarize_text(text):
    llm = OpenAI(api_key='your_openai_api_key')
    chain = LLMChain(llm=llm, prompt=StringPromptTemplate(
        input_variables=["text"],
        template="Summarize the following text: {text}"
    ))
    summary = chain.run(text=text)
    return summary

In [11]:
def create_agentic_workflow(url):
    tools = [
        Tool(name="Scrape HTML", func=scrape_html, description="Scrape projects from HTML"),
        Tool(name="Extract PDF", func=extract_pdf_text, description="Extract text from PDF"),
        Tool(name="Extract Image", func=extract_image_text, description="Extract text from image using OCR"),
        Tool(name="Summarize", func=summarize_text, description="Summarize extracted text")
    ]

    llm = OpenAI(api_key='your_openai_api_key')
    agent = LLMSingleActionAgent(llm_chain=LLMChain(llm=llm, prompt=StringPromptTemplate(
        input_variables=["input", "tools"],
        template="Given the URL {input}, choose the appropriate tool to extract project information: {tools}"
    )), allowed_tools=[tool.name for tool in tools])

    executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
    return executor.run(url)

In [12]:
def create_crew():
    scraper = Agent(
        role="Web Scraper",
        goal="Extract project information from websites",
        backstory="Expert in web scraping and data extraction",
        tools=[scrape_html, extract_pdf_text, extract_image_text]
    )

    analyzer = Agent(
        role="Data Analyzer",
        goal="Analyze and summarize extracted project information",
        backstory="Expert in data analysis and summarization",
        tools=[summarize_text]
    )

    scraping_task = Task(
        description="Scrape project information from the given URL",
        agent=scraper
    )

    analysis_task = Task(
        description="Analyze and summarize the extracted project information",
        agent=analyzer
    )

    crew = Crew(
        agents=[scraper, analyzer],
        tasks=[scraping_task, analysis_task]
    )

    return crew

def process_url_with_crew(url):
    crew = create_crew()
    result = crew.kickoff(url)
    return result

In [13]:
def main():
    print("Project Extraction and Analysis Tool")
    print("====================================")
    
    while True:
        url = input("Enter a URL to process (or 'quit' to exit): ")
        if url.lower() == 'quit':
            break
        
        print("Processing URL:", url)
        print("1. Using Agentic Workflow")
        agentic_result = create_agentic_workflow(url)
        print("Agentic Workflow Result:", agentic_result)
        
        print("\n2. Using CrewAI")
        crew_result = process_url_with_crew(url)
        print("CrewAI Result:", crew_result)
        
        print("\nProcessing complete.")
        print("------------------------------------")

if __name__ == "__main__":
    main()

Project Extraction and Analysis Tool
Processing URL: https://www.gatesfoundation.org/
1. Using Agentic Workflow


/opt/anaconda3/envs/mlx/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


TypeError: Can't instantiate abstract class StringPromptTemplate with abstract method format